# Stitch to Cloudlab using a FABRIC facility port

This notebook shows how to use a facility port to connect your FABRIC experiment to the Cloudlab testbed.

## Import the FABlib library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.



In [ ]:
slice_name = "MySlice"

node_name = 'node1'
network_name = 'net1'

facility_port='Utah-Cloudlab-Powder'
facility_port_site='UTAH'
facility_port_vlan='3101'

#Create a slice
slice = fablib.new_slice(name=slice_name)

# Create a L2 network, the network config should match the subnet used on the Cloudlab side
net = slice.add_l2network(name=network_name, subnet=IPv4Network('192.168.1.0/24'))

node = slice.add_node(name=node_name, site='UTAH')

# If choosing a FABRIC site not directly connected to the Cloudlab site, NIC_Basic cannot be used with the required L2 network
#node_iface = node.add_component(model='NIC_ConnectX_6', name="nic1").get_interfaces()[0]
node_iface = node.add_component(model='NIC_Basic', name="nic1").get_interfaces()[0]
net.add_interface(node_iface)
node_iface.set_mode('config')
# Set a specific IP address for the stitch interface on the node
node_iface.set_ip_addr(IPv4Address("192.168.1.10"))

# Create a facility port component
facility_port = slice.add_facility_port(name=facility_port, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface = facility_port.get_interfaces()[0]
net.add_interface(facility_port_interface)
                          
print(f"facility_port.get_site(): {facility_port.get_site()}")

#Submit the Request
slice.submit();

## Cloudlab configuration

Now that the FABRIC side has been allocated,and a network is constructed towards the Cloudlab facility port, we can create resources on the Cloudlab side. We will use an existing profile that allows for the selection of stitching vlan, IP subnet, and node count. The profile is available here: https://www.cloudlab.us/p/fabfed/fabfed-stitch-v2

The follwing instructions demonstrate how to use the Cloudlab Web Portal to complete this stitching experiment. Programmatic options exist through the Cloudlab API, tools, and FabFed.

# Instantiate a new instance of this profile.

Login to Cloudlab using your credentials and load the profile page linked above.

<img src="img/profile.png" width="80%"><br>

# Set profile parameters

Next, fill in the desired paramaters available in the stitching profile. The VLAN ID must match the vlan selected for the FABRIC facility port above. The IP subnet should match that subnet that was used in the FABRIC network above. You may select one or more nodes for the Cloudlab LAN segment that will be connected at layer2 to the facility port.

<img src="img/params.png" width="80%"><br>

# Select a Cloudlab site or cluster

<img src="img/select.png" width="80%"><br>

# Create the experiment and test.

Complete the steps to instantiate the profile, submit, and wait for the Cloudlab experiment to become active.

Once the experiment is active, the Cloudlab portal interface can be used to access a shell on the nodes from which you can ping the FABRIC nodes.

<img src="img/test.png" width="80%"><br>

## Ping the Cloudlab resource from your FABRIC node

Update the remote IP as needed.  Given an IP subnet, the Cloudlab profile will start assigning from the first host address.

In [ ]:
slice = fablib.get_slice(slice_name)
node = slice.get_node(name=node_name)
stdout, stderr = node.execute('ping -c 5 192.168.1.1')

## Delete the slice

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()

## Terminate the Cloudlab experiment

Use the Cloudlab portal or other tools to release any experimental resources when no longer needed.